# Fuzzy C-Means
Fuzzy C-Means (FCM), atau dikenal juga sebagai Fuzzy Isodata merupakan salah satu metode clustering yang merupakan bagian dari metode Hard K-Means. FCM menggunakan model pengelompokan fuzzy sehingga data dapat menjadi anggota dari semua kelas atau cluster terbentuk dengan derajat atau tingkat keanggotaan yang berbeda antara 0 hingga 1. Tingkat keberadaan data dalam suatu kelas atau cluster ditentukan oleh derajat keanggotaannya.

Konsep dasar FCM, pertama kali adalah menentukan pusat cluster yang akan menandai lokasi rata-rata untuk setiap cluster. Pada kondisi awal pusat cluster ini masih belum akurat. Setiap data memiliki derajat keanggotaan untuk setiap cluster. Dengan cara memperbaiki pusat cluster dan nilai keanggotaan setiap data secara berulang, maka dapat dilihat bahwa pusat cluster akan menuju lokasi yang tepat. Perulangan ini didasarkan pada minimasi fungsi objektif yang menggambarkan jarak dari titik data yang diberikan ke pusat cluster yang berbobot oleh derajat keanggotaan titik data tersebut.

Algoritmanya:
1. Input data yang akan di-cluster, 𝑋, berupa matriks berukuran 𝑛 × 𝑚 (𝑛 = jumlah sampel data, 𝑚 = atribut setiap data). 𝑋𝑖𝑗 data sampel ke-𝑖 (𝑖 = 1,2, ⋯ , 𝑛), atribut ke-𝑗 (𝑗 = 1,2, ⋯ , 𝑚).
2. Tentukan: 
    - Jumlah cluster = 𝑐 
    - Pangkat = 𝑤 
    - Maksimum iterasi = Maxlter 
    - Error terkecil yang diharapkan = 𝜉 
    - Fungsi objektif awal = 𝑃0 = 0 
    - Iterasi awal = 𝑡 = 1
3. Bangkitkan bilangan random 𝜇𝑖𝑘, 𝑖 = 1,2, ⋯ , 𝑛; 𝑘 = 1,2, ⋯ , 𝑐; sebagai elemen-elemen matriks partisi awal 𝑈.  <br>
![img3](https://i.ibb.co/G07W7vx/3.png)
4. Hitung pusat cluster ke-𝑘, 𝑉𝑘𝑗 dengan 𝑘 = 1,2, ⋯ , 𝑐 dan 𝑗 = 1,2, ⋯ , m. <br>
![img4](https://i.ibb.co/7QqQjtL/4.png)
5. Hitung fungsi objektif pada iterasi ke-𝑡, Pt. <br>
![img5](https://i.ibb.co/3RG88kX/5.png)
6. Hitung perubahan matriks partisi. <br>
![img6](https://i.ibb.co/BVXBZ1M/6.png)
7. Cek kondisi berhenti 
    - Jika: (|𝑃𝑡 – 𝑃𝑡 − 1| < 𝜉) atau (𝑡 > Maxlter), maka berhenti; 
    - Jika tidak: 𝑡 = 𝑡 + 1, ulangi langkah ke-4.

<a id="top"></a>
## Langkah-Langkah Implementasi Kodingan
1. Jalankan kodingan di [Section #1](#1).
2. Pilih data yang ingin di-ujicobakan pada [Section #2](#2). Jika ingin menggunakan data dan miu sendiri, inisialisasi keduanya dalam bentuk array. Perlu diperhatikan bahwa data wajib ada, sedangkan miu opsional karena terdapat kodingan yang dapat generate miu secara dinamis.
3. Kemudian buat objek Fuzzy C-Means nya sebagaimana pada [Section #3](#3) dan masukkan argumen yang ingin dicoba. Penjelasan parameter dari kelas FCM adalah sebagai berikut:
    - data => data yang diujicoba (wajib)
    - c => jumlah cluster (default=2)
    - w => tingkat fuzzy/pangkat (default=2)
    - max_iter => iterasi maksimum yang dilakukan (default=100)
    - e => error terkecil yang diharapkan (default=0.01)
    - p => fungsi objektif awal (default=0)
    - t => iterasi awal (default=1)
    - miu => bobot derajat keanggotaan dari data (default=None)
4. Jika objek FCM sudah dibuat, kita dapat mengecek atributnya ([Section #4](#4)), yaitu:
    - miu => bobot derajat
    - part_u => matriks partisi
    - c_center => pusat cluster
    - get_cluster_result() => hasil dari pengelompokan clusternya

<a id="1"></a>
## Section #1

In [1]:
# import library
from random import uniform
import numpy as np

In [2]:
class FCM:
    '''Kelas Fuzzy C-Means'''
    
    c_center = None
    part_u = None
    
    # fungsi konstruktor
    def __init__(self, data, c=2, w=2, max_iter=100, e=0.01, p=0, t=1, miu=None):
        if len(data[0]) < 2:
            raise FCMError("Masa fiturnya dikit sih!")
        if self.check_diff_len(data):
            raise FCMError("Ukuran data tidak sama!")
        if miu is not None and self.check_diff_len(miu):
            raise FCMError("Ukuran miu tidak sama!")
        if miu is not None and len(miu) != len(data):
            raise FCMError("Jumlah sampel dari data dan miu tidak selaras!")
        if c < 2:
            raise FCMError("Jumlah clustering tidak valid!")
        self.data = data
        self.n = len(data) # banyak sampel
        self.c = c if miu is None else len(miu[0]) # jumlah cluster
        self.w = w # tingkat keabuan 
        self.max_iter = max_iter # maksimum iterasi
        self.e = e # error
        self.p = [0] # kumpulan fungsi objektif
        self.t = t # iterasi
        self.miu = miu if miu is not None else self.generate_miu() # miu atau bobot
        self.training()
        
    # fungsi untuk mengecek apakah ukuran tiap baris datanya konsisten
    def check_diff_len(self, lists):
        it = iter(lists)
        the_len = len(next(it))
        if not all(len(l) == the_len for l in it):
             return True
        else:
            return False
        
    # fungsi untuk random nilai miu jika pengguna tidak menentukannya sendiri
    def generate_miu(self):
        miu = np.zeros((self.n, self.c))
        for i in range(self.n):
            for j in range(self.c):
                if j == 0:
                    miu[i,j] = round(uniform(0.1, 0.9), 2)
                else:
                    left = 1 - round(sum(miu[i,:j]),2)
                    if j == self.c-1:
                        miu[i,j] = round(left, 2)
                    else:
                        miu[i,j] = round(uniform(0.1, left/(self.c-j)), 2)
        return miu
    
    # fungsi untuk melatih berdasar data dan miu
    def training(self):
        while True:
            # miu kuadrat
            miu_w = self.miu ** self.w
            # kalikan nilai x dengan miu kuadratnya
            cross_xc = []
            for i in range(self.c):
                cross_xc.append((self.data.transpose() * miu_w[:, i]).transpose())
            # hitung total miu kuadrat
            sum_miu_w = np.sum(miu_w, axis=0)
            # hitung total dari perkalian tiap cluster tadi
            sum_cross_xc = np.sum(cross_xc, axis=1)
            # cari titik pusatnya
            c_center = (sum_cross_xc.transpose() / sum_miu_w).transpose()
            # hitung fungsi objektif
            obj_fun = np.zeros((self.n, self.c))
            for i in range(self.n):
                for j in range(self.c):
                    obj_fun[i,j] = sum((self.data[i]-c_center[j])**2)*miu_w[i][j]
            # hitung total fungsi objektifnya
            sum_obj_fun = np.sum(obj_fun)
            self.p.append(sum_obj_fun)
            # jika lanjut, cari matriks partisi u
            part_u = np.zeros((self.n, self.c))
            for i in range(self.n):
                for j in range(self.c):
                    part_u[i,j] = sum((data[i]-c_center[j])**2)**-1/(self.w-1)
            # hitung total dari tiap baris partisi
            lt = np.sum(part_u, axis=1)
            # hitung miu baru
            new_miu = (part_u.transpose() / lt).transpose()
            # lakukan t selanjutnya jika belum max_iter
            if abs(self.p[self.t] - self.p[self.t-1]) <= self.e or self.t == self.max_iter:
                break
            else:
                self.miu = new_miu
                self.t += 1
        # ambil nilai cluster dan matriks partisi u saat iterasi terakhir
        self.c_center = c_center
        self.part_u = part_u
        print(f"Berakhir saat iterasi ke-{self.t}")
        
    # fungsi untuk mendapatkan hasil pengelompokan clustering        
    def get_cluster_result(self):
        result = {}
        cluster_result = (np.argmax(self.part_u, axis=1) + 1)
        for i in range(self.n):
            if cluster_result[i] not in result:
                result[cluster_result[i]] = [i+1]
            else:
                result[cluster_result[i]].append(i+1)
        result = dict(sorted(result.items())) # {key:result[key] for key in sorted(result.keys())}
        print(result)
        return result
        
class FCMError(Exception):
    pass

[Kembali ke atas](#top)
<a id="2"></a>
## Section #2
Coba Tambah Data Trainingnya Yok !

Referensi data dan miu di bawah ini:
- [Data YT](https://www.youtube.com/watch?v=7b7vVhSaFFs)
- [Data Excel](https://berajah.if.unram.ac.id/pluginfile.php/19093/mod_resource/content/1/htiung%20FCM%20revisi.xlsx)
- [Data Beasiswa](https://media.neliti.com/media/publications/277582-implementasi-fuzzy-c-means-clustering-da-3afa5ba1.pdf)

In [3]:
# data YT
data = np.array([
    [1, 3],
    [3, 3],
    [4, 3],
    [5, 3],
    [1, 2],
    [4, 2],
    [1, 1],
    [2, 1],
    [5, 2],
    [2, 4]
])

miuw = np.array([
    [0.28, 0.5 , 0.22],
    [0.74, 0.12, 0.14],
    [0.02, 0.83, 0.15],
    [0.57, 0.02, 0.41],
    [0.23, 0.15, 0.62],
    [0.26, 0.52, 0.22],
    [0.68, 0.31, 0.01],
    [0.85, 0.12, 0.03],
    [0.19, 0.39, 0.42],
    [0.46, 0.35, 0.19]
])

In [ ]:
# data excel
data = np.array([
    [10,  8],
    [ 4,  5],
    [ 2,  3],
    [ 9,  7],
    [ 0,  1]
])

miuw = np.array([
    [0.3, 0.7],
    [0.6, 0.4],
    [0.7, 0.3],
    [0.4, 0.6],
    [0.8, 0.2]
])

In [3]:
# data artikel beasiswa
data = np.array([
    [1. , 0.5, 0.5, 1. , 0.5],
    [1. , 0.5, 0.5, 1. , 0. ],
    [1. , 0.5, 0. , 1. , 0.5],
    [1. , 1. , 0. , 0.5, 0.5],
    [1. , 0. , 1. , 0.5, 0.5],
    [1. , 0.5, 0. , 1. , 0.5],
    [1. , 0. , 0.5, 1. , 0.5],
    [1. , 0. , 0. , 0. , 0.5],
    [1. , 1. , 0. , 0. , 0.5],
    [0. , 0.5, 1. , 1. , 0.5],
    [1. , 0.5, 0.5, 1. , 0.5],
    [1. , 0.5, 1. , 1. , 0.5],
    [1. , 0.5, 1. , 1. , 0.5],
    [0. , 0. , 0.5, 0.5, 0.5]
])

miuw = np.array([
    [0.18 , 0.82 ],
    [0.323, 0.677],
    [0.673, 0.327],
    [0.834, 0.166],
    [0.273, 0.727],
    [0.673, 0.327],
    [0.226, 0.774],
    [0.696, 0.304],
    [0.778, 0.222],
    [0.326, 0.674],
    [0.18 , 0.82 ],
    [0.125, 0.875],
    [0.125, 0.875],
    [0.458, 0.542]
])

[Kembali ke atas](#top)
<a id="3"></a>
## Section #3
Coba Test Buat Objeknya Yok !

In [4]:
# fcm YT
obj = FCM(data, e=10**-4, miu=miuw)

Berakhir saat iterasi ke-18


In [ ]:
# fcm excel
obj = FCM(data, miu=miuw, e=0.1)

In [4]:
# fcm artikel beasiswa
obj = FCM(data, miu=miuw, max_iter=50)

Berakhir saat iterasi ke-2


Tes Generate Miunya Yok !

In [ ]:
obj.generate_miu()

[Kembali ke atas](#top)
<a id="4"></a>
## Section #4
Cek Atributnya Yok !

In [5]:
np.round(obj.miu, 3)

array([[0.18 , 0.82 ],
       [0.323, 0.677],
       [0.672, 0.328],
       [0.834, 0.166],
       [0.273, 0.727],
       [0.672, 0.328],
       [0.226, 0.774],
       [0.696, 0.304],
       [0.778, 0.222],
       [0.326, 0.674],
       [0.18 , 0.82 ],
       [0.125, 0.875],
       [0.125, 0.875],
       [0.458, 0.542]])

In [6]:
np.round(obj.part_u, 2)

array([[ 2.65, 12.12],
       [ 1.63,  3.43],
       [ 3.91,  1.91],
       [ 4.78,  0.95],
       [ 0.91,  2.42],
       [ 3.91,  1.91],
       [ 1.43,  4.9 ],
       [ 1.59,  0.69],
       [ 2.06,  0.59],
       [ 0.55,  1.14],
       [ 2.65, 12.12],
       [ 1.  ,  7.03],
       [ 1.  ,  7.03],
       [ 0.78,  0.92]])

In [7]:
np.round(obj.c_center, 2)

array([[0.91, 0.57, 0.13, 0.53, 0.48],
       [0.87, 0.37, 0.69, 0.9 , 0.46]])

In [8]:
cluster_result = obj.get_cluster_result()

{1: [3, 4, 6, 8, 9], 2: [1, 2, 5, 7, 10, 11, 12, 13, 14]}


[Kembali ke atas](#top)
### Hitung Akurasi (Khusus Beasiswa)

In [9]:
target = {
    1: [10, 14],
    2: [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13]
}

In [10]:
# list bener dan salah ditebak
correct, wrong = [], []
n = len(data)

# looping untuk cek yang mana yang bener dan salah
for k in target:
    for v in target[k]:
        if v in cluster_result[k]:
            correct.append(v)
        else:
            wrong.append(v)
            
# akurasi
accuracy = (len(correct)/n)*100
print(f'Akurasi: {accuracy}%')

Akurasi: 50.0%
